In [31]:
# amadeus_list_by_city_df.py
"""
Fonctions minimalistes et robustes pour:
1) Obtenir un token sandbox Amadeus
2) Lister des hôtels par code ville (IATA) via Hotel List (by-city)
3) Convertir la réponse en DataFrame pandas
"""

from __future__ import annotations
import os, time, httpx, pandas as pd
from dotenv import load_dotenv

AUTH_URL = "https://test.api.amadeus.com/v1/security/oauth2/token"
LIST_BY_CITY_URL = "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city"

load_dotenv()

def _post(url: str, data: dict, timeout: int = 20) -> httpx.Response:
    last_exc = None
    for attempt in range(3):
        try:
            return httpx.post(
                url,
                data=data,
                headers={"Content-Type": "application/x-www-form-urlencoded", "Accept": "application/json"},
                timeout=timeout,
            )
        except httpx.HTTPError as e:
            last_exc = e
            time.sleep(0.5 * (attempt + 1))
    raise last_exc

def _get(url: str, headers: dict, params: dict, timeout: int = 30) -> httpx.Response:
    last_exc = None
    for attempt in range(3):
        try:
            return httpx.get(url, headers=headers, params=params, timeout=timeout)
        except httpx.HTTPError as e:
            last_exc = e
            time.sleep(0.5 * (attempt + 1))
    raise last_exc

def get_token() -> str:
    client_id = os.getenv("AMADEUS_CLIENT_ID")
    client_secret = os.getenv("AMADEUS_CLIENT_SECRET")
    if not client_id or not client_secret:
        raise RuntimeError("AMADEUS_CLIENT_ID / AMADEUS_CLIENT_SECRET manquants dans .env")

    resp = _post(AUTH_URL, {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
    })
    resp.raise_for_status()
    token = resp.json().get("access_token")
    if not token:
        raise RuntimeError("Token introuvable dans la réponse OAuth.")
    return token

def list_hotels_by_city(token: str, city_code: str) -> list[dict]:
    if not city_code or len(city_code) != 3:
        raise ValueError("city_code doit être un code IATA à 3 lettres (ex: 'PAR').")
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    params = {"cityCode": city_code.upper()}

    resp = _get(LIST_BY_CITY_URL, headers=headers, params=params)
    resp.raise_for_status()
    return resp.json().get("data", [])

def hotels_to_dataframe(hotels: list[dict]) -> pd.DataFrame:
    """Aplatis quelques champs utiles dans un DataFrame."""
    rows = []
    for h in hotels:
        addr = h.get("address") or {}
        geo = h.get("geoCode") or {}
        rows.append({
            "hotelId": h.get("hotelId"),
            "name": h.get("name"),
            "chainCode": h.get("chainCode"),
            "city": addr.get("cityName"),
            "country": addr.get("countryCode"),
            "postalCode": addr.get("postalCode"),
            "latitude": geo.get("latitude"),
            "longitude": geo.get("longitude"),
        })
    return pd.DataFrame(rows)

if __name__ == "__main__":
    tok = get_token()
    hotels = list_hotels_by_city(tok, "PAR")
    df = hotels_to_dataframe(hotels)
    print(df.head(10))


    hotelId                               name chainCode   city country  \
0  BRPARVDB          RENAISSANCE PARIS VENDOME        BR  PARIS      FR   
1  RTPARMAI       IBIS PARIS TOUR MONTPARNASSE        RT  PARIS      FR   
2  RTPARFAY            MERCURE PARIS LAFAYETTE        RT  PARIS      FR   
3  RTPAREIF       MERCURE PARIS TOUR EIFFEL 4*        RT  PARIS      FR   
4  FGPARPAL         HOTEL PRINCE ALBERT LOUVRE        FG  PARIS      FR   
5  BWPAR789             BW PREMIER FAUBOURG 88        BW  PARIS      FR   
6  BWPAR160         BEST WESTERN GAILLON OPERA        BW  PARIS      FR   
7  HNPARKGU  HN TEST PROPERTY1 FOR E2E TESTING        HN  Paris      FR   
8  XKPAR120                      DIAMOND HOTEL        XK  Paris      FR   
9  CHPARH24                    HOTEL AMBASSADE        CH  PARIS      FR   

  postalCode  latitude  longitude  
0      75001  48.86195    2.33592  
1      75015  48.84333    2.32019  
2      75009  48.87281    2.34493  
3      75015  48.85052    2.29

In [28]:
# hotel_offers_all_per_hotel.py
from __future__ import annotations
import os
from datetime import date, timedelta
from typing import List, Dict, Any
import httpx
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

AUTH_URL   = "https://test.api.amadeus.com/v1/security/oauth2/token"
OFFERS_URL = "https://test.api.amadeus.com/v3/shopping/hotel-offers"

def get_token() -> str:
    r = httpx.post(
        AUTH_URL,
        data={
            "grant_type": "client_credentials",
            "client_id": os.getenv("AMADEUS_CLIENT_ID"),
            "client_secret": os.getenv("AMADEUS_CLIENT_SECRET"),
        },
        headers={"Content-Type": "application/x-www-form-urlencoded", "Accept": "application/json"},
        timeout=20,
    )
    r.raise_for_status()
    tok = r.json().get("access_token")
    if not tok:
        raise RuntimeError("Token introuvable")
    return tok

def get_offers_all(token: str, hotel_ids: List[str], checkin: str, checkout: str, adults: int = 2, lang: str = "FR") -> List[Dict[str, Any]]:
    """Récupère toutes les offres renvoyées par v3 pour les IDs fournis (sans tronquer côté client)."""
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    params = {
        "hotelIds": ",".join(hotel_ids),
        "adults": str(adults),
        "roomQuantity": "1",
        "checkInDate": checkin,
        "checkOutDate": checkout,
        "currency": "EUR",
        "lang": lang,  # descriptions FR si dispo
        # Si besoin, tu peux ajouter "page[limit]" et "page[offset]" si Amadeus te retourne beaucoup de résultats.
    }
    r = httpx.get(OFFERS_URL, headers=headers, params=params, timeout=60)
    if r.status_code >= 400:
        print("OFFERS ERROR:", r.status_code, r.text[:400])
    r.raise_for_status()
    return r.json().get("data", []) or []

def offers_to_df(items: List[Dict[str, Any]]) -> pd.DataFrame:
    """1 ligne = 1 offre; on garde les champs clés (prix, chambre, board, paymentType, annulation)."""
    from datetime import date as _d
    rows = []
    for it in items:
        h = it.get("hotel") or {}
        hid = h.get("hotelId")
        hname = h.get("name")
        city = h.get("cityCode")
        lat = h.get("latitude"); lon = h.get("longitude")

        for of in it.get("offers") or []:
            price = (of.get("price") or {})
            total = price.get("total"); cur = price.get("currency")
            ci = of.get("checkInDate"); co = of.get("checkOutDate")

            # prix / nuit
            nights = nightly = None
            try:
                d0 = _d.fromisoformat(ci); d1 = _d.fromisoformat(co)
                nights = max((d1 - d0).days, 1)
                nightly = round(float(total)/nights, 2) if total else None
            except Exception:
                pass

            room = of.get("room") or {}
            t_est = room.get("typeEstimated") or {}
            room_desc = (room.get("description") or {}).get("text")
            board = of.get("boardType")  # ex: ROOM_ONLY, BREAKFAST, HALF_BOARD...

            # pas de nom d’OTA : mais on a le mode de paiement
            payment = of.get("paymentType")  # ex: PAY_AT_HOTEL / PREPAID

            # un petit résumé d’annulation (si dispo)
            policies = (of.get("policies") or {}).get("cancellations") or []
            cancel_summary = None
            if policies:
                # on prend la première règle comme résumé
                p0 = policies[0]
                cancel_summary = f"{p0.get('description', {}).get('text', '')}".strip() or str(p0)[:140]

            rows.append({
                # Prix d'abord
                "price_total": float(total) if total else None,
                "currency": cur,
                "price_per_night": nightly,
                "nights": nights,

                # Hôtel
                "hotelId": hid,
                "hotel_name": hname,
                "city": city,
                "latitude": lat,
                "longitude": lon,

                # Séjour
                "checkIn": ci,
                "checkOut": co,

                # Offre / chambre
                "offerId": of.get("id"),
                "room_type": room.get("type"),
                "room_category": t_est.get("category"),
                "room_beds": t_est.get("beds"),
                "bed_type": t_est.get("bedType"),
                "room_desc": room_desc,
                "boardType": board,
                "paymentType": payment,
                "cancel_policy": cancel_summary,
                # Pas de champ "site/OTA" dans la réponse Self-Service.
            })
    return pd.DataFrame(rows)

if __name__ == "__main__":
    token = get_token()

    ci = (date.today() + timedelta(days=20)).isoformat()
    co = (date.today() + timedelta(days=22)).isoformat()

    # prends 3–10 hotelIds qui renvoient des offres en sandbox
    hotel_ids = ["ARNCEACH", "BWNCE645", "MDNCEMER"]

    items = get_offers_all(token, hotel_ids, ci, co, adults=2, lang="FR")
    df = offers_to_df(items)

    pd.set_option("display.max_columns", None)
    print(df.head(20))
    # df.to_csv("all_offers_per_hotel.csv", index=False)


   price_total currency  price_per_night  nights   hotelId  \
0      1092.12      EUR           546.06       2  ARNCEACH   
1       201.76      EUR           100.88       2  BWNCE645   
2       971.92      EUR           485.96       2  MDNCEMER   

                             hotel_name city  latitude  longitude     checkIn  \
0             AC Hotel by Marriott Nice  NCE  43.69297    7.25242  2025-10-04   
1  Best Western Premier Hotel Roosevelt  NCE  43.69892    7.26585  2025-10-04   
2                      Le Meridien Nice  NCE  43.69495    7.26551  2025-10-04   

     checkOut     offerId room_type  room_category  room_beds bed_type  \
0  2025-10-06  WCP43U2192       AP7  STANDARD_ROOM          1     KING   
1  2025-10-06  GXQAQ6YDD2       A1D  STANDARD_ROOM          1   DOUBLE   
2  2025-10-06  ROM5B43M5Q       REG  STANDARD_ROOM          1     KING   

                                           room_desc boardType paymentType  \
0  Prepay Non-refundable Non-changeable, prepay i..

In [29]:
df

,price_total,currency,price_per_night,nights,hotelId,hotel_name,city,latitude,longitude,checkIn,checkOut,offerId,room_type,room_category,room_beds,bed_type,room_desc,boardType,paymentType,cancel_policy
0,1092.12,EUR,546.06,2,ARNCEACH,AC Hotel by Marriott Nice,NCE,43.69297,7.25242,2025-10-04,2025-10-06,WCP43U2192,AP7,STANDARD_ROOM,1,KING,"Prepay Non-refundable Non-changeable, prepay i...",None,None,NON-REFUNDABLE RATE
1,201.76,EUR,100.88,2,BWNCE645,Best Western Premier Hotel Roosevelt,NCE,43.69892,7.26585,2025-10-04,2025-10-06,GXQAQ6YDD2,A1D,STANDARD_ROOM,1,DOUBLE,"MULTI NIGHT STAY PROMOTION\n1 DOUBLE BED,NSMK,...",None,None,NON-REFUNDABLE RATE
2,971.92,EUR,485.96,2,MDNCEMER,Le Meridien Nice,NCE,43.69495,7.26551,2025-10-04,2025-10-06,ROM5B43M5Q,REG,STANDARD_ROOM,1,KING,"Flexible Rate\nClassic Double Room, 1 King, 25...",None,None,"{'numberOfNights': 1, 'deadline': '2025-10-01T..."


In [37]:
# city_to_all_offers_df.py
from __future__ import annotations
import os, time
from datetime import date, timedelta
from typing import List, Dict, Any
import httpx
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

AUTH_URL = "https://test.api.amadeus.com/v1/security/oauth2/token"
LIST_BY_CITY_URL = "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city"
OFFERS_URL = "https://test.api.amadeus.com/v3/shopping/hotel-offers"

# -------- helpers HTTP (reprise de ton code) --------
def _post(url: str, data: dict, timeout: int = 20) -> httpx.Response:
    last_exc = None
    for attempt in range(3):
        try:
            return httpx.post(
                url,
                data=data,
                headers={"Content-Type": "application/x-www-form-urlencoded", "Accept": "application/json"},
                timeout=timeout,
            )
        except httpx.HTTPError as e:
            last_exc = e
            time.sleep(0.5 * (attempt + 1))
    raise last_exc

def _get(url: str, headers: dict, params: dict, timeout: int = 30) -> httpx.Response:
    last_exc = None
    for attempt in range(3):
        try:
            return httpx.get(url, headers=headers, params=params, timeout=timeout)
        except httpx.HTTPError as e:
            last_exc = e
            time.sleep(0.5 * (attempt + 1))
    raise last_exc

# -------- auth --------
def get_token() -> str:
    client_id = os.getenv("AMADEUS_CLIENT_ID")
    client_secret = os.getenv("AMADEUS_CLIENT_SECRET")
    if not client_id or not client_secret:
        raise RuntimeError("AMADEUS_CLIENT_ID / AMADEUS_CLIENT_SECRET manquants dans .env")
    resp = _post(AUTH_URL, {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
    })
    resp.raise_for_status()
    token = resp.json().get("access_token")
    if not token:
        raise RuntimeError("Token introuvable dans la réponse OAuth.")
    return token

# -------- liste d’hôtels (by-city) --------
def list_hotels_by_city(token: str, city_code: str) -> list[dict]:
    if not city_code or len(city_code) != 3:
        raise ValueError("city_code doit être un code IATA à 3 lettres (ex: 'PAR').")
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    params = {"cityCode": city_code.upper()}
    resp = _get(LIST_BY_CITY_URL, headers=headers, params=params)
    resp.raise_for_status()
    return resp.json().get("data", [])

# -------- offres (v3) à partir d’une liste d’hotelId --------
def _offers_call(token: str, hotel_ids: List[str], checkin: str, checkout: str, adults: int = 2, lang: str = "FR") -> list[dict]:
    headers = {"Authorization": f"Bearer {token}", "Accept": "application/json"}
    params = {
        "hotelIds": ",".join(hotel_ids),
        "adults": str(adults),
        "roomQuantity": "1",
        "checkInDate": checkin,
        "checkOutDate": checkout,
        "currency": "EUR",
        "lang": lang,  # descriptions FR si dispo
    }
    r = _get(OFFERS_URL, headers=headers, params=params, timeout=60)
    if r.status_code >= 400:
        # debug court, on laisse passer en haut
        print(f"[WARN] OFFERS {r.status_code} ids={hotel_ids[:3]}... msg={r.text[:200]}")
    r.raise_for_status()
    return r.json().get("data", []) or []

def get_all_offers_from_city(token: str, city_code: str, checkin: str, checkout: str,
                             adults: int = 2, limit_ids: int = 20, chunk_size: int = 10) -> list[dict]:
    """
    1) by-city -> récupère des hotelId (tronqués à limit_ids pour rester rapide)
    2) v3 -> récupère toutes les offres par paquets de 'chunk_size'
    Stratégie simple: si un chunk 400, on tombe en mode unitaire pour ignorer les ID foireux.
    """
    hotels = list_hotels_by_city(token, city_code)
    hotel_ids = [h.get("hotelId") for h in hotels][:limit_ids]

    all_items: list[dict] = []
    for i in range(0, len(hotel_ids), chunk_size):
        pack = hotel_ids[i:i+chunk_size]
        try:
            all_items.extend(_offers_call(token, pack, checkin, checkout, adults=adults))
        except httpx.HTTPStatusError:
            # fallback unitaire pour isoler les IDs qui plantent
            for hid in pack:
                try:
                    all_items.extend(_offers_call(token, [hid], checkin, checkout, adults=adults))
                except httpx.HTTPStatusError:
                    # on ignore l’ID qui casse, sandbox oblige
                    continue
    return all_items

# -------- flatten -> DataFrame (1 ligne = 1 offre) --------
def offers_to_df(items: list[dict]) -> pd.DataFrame:
    from datetime import date as _d
    rows = []
    for it in items:
        h = it.get("hotel") or {}
        hid = h.get("hotelId")
        hname = h.get("name")
        city = h.get("cityCode")
        lat = h.get("latitude"); lon = h.get("longitude")

        for of in it.get("offers") or []:
            price = (of.get("price") or {})
            total = price.get("total"); cur = price.get("currency")
            ci = of.get("checkInDate"); co = of.get("checkOutDate")
            nights = nightly = None
            try:
                d0 = _d.fromisoformat(ci); d1 = _d.fromisoformat(co)
                nights = max((d1 - d0).days, 1)
                nightly = round(float(total)/nights, 2) if total else None
            except Exception:
                pass

            room = of.get("room") or {}
            t_est = room.get("typeEstimated") or {}
            room_desc = (room.get("description") or {}).get("text")
            board = of.get("boardType")
            payment = of.get("paymentType")
            policies = (of.get("policies") or {}).get("cancellations") or []
            cancel_summary = None
            if policies:
                p0 = policies[0]
                cancel_summary = (p0.get("description") or {}).get("text") or str(p0)[:140]

            rows.append({
                # prix en premier
                "price_total": float(total) if total else None,
                "currency": cur,
                "price_per_night": nightly,
                "nights": nights,

                # hôtel
                "hotelId": hid,
                "hotel_name": hname,
                "city": city,
                "latitude": lat,
                "longitude": lon,

                # séjour
                "checkIn": ci,
                "checkOut": co,

                # offre/chambre
                "offerId": of.get("id"),
                "room_type": room.get("type"),
                "room_category": t_est.get("category"),
                "room_beds": t_est.get("beds"),
                "bed_type": t_est.get("bedType"),
                "room_desc": room_desc,
                "boardType": board,
                "paymentType": payment,
                "cancel_policy": cancel_summary,
            })
    return pd.DataFrame(rows)

# -------- exécution --------
if __name__ == "__main__":
    token = get_token()

    # Dates “sandbox-friendly” (J+20 à J+60)
    ci = (date.today() + timedelta(days=25)).isoformat()
    co = (date.today() + timedelta(days=27)).isoformat()

    CITY = "PAR"        # <- change la ville ici (ex: "PAR", "LYS", "TLS")
    LIMIT_IDS = 1000      # <- on limite le nb d’hôtels pour éviter les 400 en série
    CHUNK = 10          # <- envoi des IDs par paquets

    items = get_all_offers_from_city(token, CITY, ci, co, adults=2, limit_ids=LIMIT_IDS, chunk_size=CHUNK)
    df = offers_to_df(items)

    pd.set_option("display.max_columns", None)
    print(df.head(20))
    # df.to_csv(f"offers_{CITY}_{ci}_{co}.csv", index=False)


    price_total currency  price_per_night  nights   hotelId  \
0       1242.32      EUR           621.16       2  BRPARVDB   
1        324.60      EUR           162.30       2  BWPAR160   
2        426.32      EUR           213.16       2  BWPAR789   
3        290.00      USD           145.00       2  FGPARPAL   
4         92.00      EUR            46.00       2  HNPARKGU   
5       1280.30      EUR           640.15       2  RTPAREIF   
6        181.60      EUR            90.80       2  RTPARMAI   
7       1740.00      EUR           870.00       2  XKPAR120   
8        279.72      EUR           139.86       2  BWPAR728   
9         56.00      EUR            28.00       2  VPPAR4JH   
10       220.00      EUR           110.00       2  RTPAR673   
11       158.08      EUR            79.04       2  RTPARIBI   
12       483.80      EUR           241.90       2  HIPARA50   
13        60.00      EUR            30.00       2  HNPARWHZ   
14       200.00      EUR           100.00       2  HNPA

In [47]:
df.sort_values(by="price_per_night", ascending=True) 

,price_total,currency,price_per_night,nights,hotelId,hotel_name,city,latitude,longitude,checkIn,checkOut,offerId,room_type,room_category,room_beds,bed_type,room_desc,boardType,paymentType,cancel_policy
34,40.00,EUR,20.00,2,HNPARSPC,Test property for Azure Migration HOS and HCR ...,PAR,48.85315,2.34513,2025-10-09,2025-10-11,YEOC19979B,A**,None,NaN,None,This is the best leisure rate plan suitable fo...,None,None,None
21,40.00,EUR,20.00,2,VPPARSTQ,L'HOTEL ETAIT VRAIMENT BON,PAR,0.00000,0.00000,2025-10-09,2025-10-11,8QTN3Z11VF,3BT,None,NaN,None,description added manually\nCITY XL SGL,None,None,None
9,56.00,EUR,28.00,2,VPPAR4JH,HOTEL SPLENDID ETOILE,PAR,0.00000,0.00000,2025-10-09,2025-10-11,GM2PSU4X5V,0BG,None,NaN,None,This is the best leisure rate plan\nSHORT,None,None,None
13,60.00,EUR,30.00,2,HNPARWHZ,Test Property from DCP 4,PAR,14.00000,15.00000,2025-10-09,2025-10-11,4OJ1E09JYT,A**,None,NaN,None,This is the best rate plan\nRoom with lake view,BREAKFAST,None,None
18,80.00,EUR,40.00,2,OIPAR07J,LK TEST PROPERTY FOR DRI CLOUD MIGRATION,PAR,48.85840,2.29450,2025-10-09,2025-10-11,8UNRH3DXIS,N1D,None,1.0,DOUBLE,FLASH SALE RATE\nMCK,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,1548.52,EUR,774.26,2,ICPARICB,HY Marriott Savannah Plant Riverside District,PAR,48.87031,2.32992,2025-10-09,2025-10-11,IFYHD9CY54,*RH,STANDARD_ROOM,NaN,None,BOOK EARLY ADVANCE PURCHASE\nClassic Room When...,ROOM_ONLY,None,{'policyType': 'CANCELLATION'}
89,1591.52,EUR,795.76,2,MCPARDTM,Paris Marriott Champs Elysees Hotel,PAR,48.87077,2.30446,2025-10-09,2025-10-11,A3CJ4YP0MY,AP7,DELUXE_ROOM,1.0,DOUBLE,"Prepay Non-refundable Non-changeable, prepay i...",None,None,NON-REFUNDABLE RATE
7,1740.00,EUR,870.00,2,XKPAR120,Diamond Hotel,PAR,48.86831,2.31222,2025-10-09,2025-10-11,U07V0V4W17,A1K,DELUXE_ROOM,1.0,KING,Public rate\nDeluxe King Room,None,None,None
62,1760.00,EUR,880.00,2,VPPARLVM,TEST PROPERTY FOR VPG2,PAR,0.00000,0.00000,2025-10-09,2025-10-11,UWS6YAI3BH,C**,None,NaN,None,This is room has sea view\n1 BEDROOM GARDEN VILLA,BREAKFAST,None,None


In [ ]:
df[df["hotel_name"].str.contains("Saint Germain Des", case=False, na=False)] 


,price_total,currency,price_per_night,nights,hotelId,hotel_name,city,latitude,longitude,checkIn,checkOut,offerId,room_type,room_category,room_beds,bed_type,room_desc,boardType,paymentType,cancel_policy
